In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency, norm
from data_viz import *

In [2]:
data = pd.read_csv("data/animal_center.csv")
data

,animal_id,name,animal_type,sex_intake,sex_outcome,breed,color,date_of_birth,found_location,intake_datetime,outcome_datetime,intake_type,intake_condition,outcome_type,outcome_subtype,age_upon_intake(years),age_upon_outcome(years),duration(days)
0,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research in Austin (TX),2014-03-07 14:26:00,2014-03-08 17:10:00,Public Assist,Normal,Return to Owner,NaN,6.7,6.7,1.0
1,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research Blvd in Austin (TX),2014-12-19 10:21:00,2014-12-20 16:35:00,Public Assist,Normal,Return to Owner,NaN,7.4,7.4,1.0
2,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,Colony Creek And Hunters Trace in Austin (TX),2017-12-07 14:07:00,2017-12-07 00:00:00,Stray,Normal,Return to Owner,NaN,10.4,10.4,-1.0
3,A047759,Oreo,Dog,Neutered Male,Neutered Male,Dachshund,Tricolor,2004-04-02,Austin (TX),2014-04-02 15:55:00,2014-04-07 15:12:00,Owner Surrender,Normal,Transfer,Partner,10.0,10.0,4.0
4,A134067,Bandit,Dog,Neutered Male,Neutered Male,Shetland Sheepdog,Brown/White,1997-10-16,12034 Research Blvd in Austin (TX),2013-11-16 09:02:00,2013-11-16 11:54:00,Public Assist,Injured,Return to Owner,NaN,16.1,16.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157648,A894260,Sassy,Cat,Intact Female,NaN,Domestic Shorthair,Torbie,NaN,Austin (TX),2023-12-05 16:56:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN
157649,A894262,Diamond,Cat,Intact Female,NaN,Domestic Shorthair,Torbie,NaN,Austin (TX),2023-12-05 16:56:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN
157650,A894263,Ruby,Cat,Intact Female,NaN,Domestic Shorthair,Tortie,NaN,Austin (TX),2023-12-05 16:56:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN
157651,A894265,Gizmo,Cat,Intact Male,NaN,Domestic Shorthair,Orange Tabby,NaN,Austin (TX),2023-12-05 16:56:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN


## **Hypothesis4:**
Does age influence the probability of adoption for animals?

$H0$: There is no significant relationship between animals' age and their likelihood of adoption.\
$H1$: There is a significant relationship between animals' age and their likelihood of adoption.

In [3]:
age_adoptiion_research = data[data["age_upon_outcome(years)"] >= 0][["animal_id", "outcome_type", "age_upon_outcome(years)"]].reset_index(drop=True)
age_adoptiion_research

,animal_id,outcome_type,age_upon_outcome(years)
0,A006100,Return to Owner,6.7
1,A006100,Return to Owner,7.4
2,A006100,Return to Owner,10.4
3,A047759,Transfer,10.0
4,A134067,Return to Owner,16.1
...,...,...,...
156628,A894012,Transfer,0.0
156629,A894054,Rto-Adopt,1.0
156630,A894102,Return to Owner,0.9
156631,A894181,Return to Owner,2.0


In [4]:
age_adoptiion_research.loc[:, "age_group"] = pd.cut(age_adoptiion_research["age_upon_outcome(years)"],
                                                    bins=[-0.01, 2, 7, float('inf')],
                                                    labels=["Young", "Adult", "Senior"])

age_adoptiion_research

,animal_id,outcome_type,age_upon_outcome(years),age_group
0,A006100,Return to Owner,6.7,Adult
1,A006100,Return to Owner,7.4,Senior
2,A006100,Return to Owner,10.4,Senior
3,A047759,Transfer,10.0,Senior
4,A134067,Return to Owner,16.1,Senior
...,...,...,...,...
156628,A894012,Transfer,0.0,Young
156629,A894054,Rto-Adopt,1.0,Young
156630,A894102,Return to Owner,0.9,Young
156631,A894181,Return to Owner,2.0,Young


In [5]:
age_adoptiion_research["adoption_status"] = np.where(age_adoptiion_research["outcome_type"] == "Adoption", "Adopted", "Not Adopted")
age_adoptiion_research

,animal_id,outcome_type,age_upon_outcome(years),age_group,adoption_status
0,A006100,Return to Owner,6.7,Adult,Not Adopted
1,A006100,Return to Owner,7.4,Senior,Not Adopted
2,A006100,Return to Owner,10.4,Senior,Not Adopted
3,A047759,Transfer,10.0,Senior,Not Adopted
4,A134067,Return to Owner,16.1,Senior,Not Adopted
...,...,...,...,...,...
156628,A894012,Transfer,0.0,Young,Not Adopted
156629,A894054,Rto-Adopt,1.0,Young,Not Adopted
156630,A894102,Return to Owner,0.9,Young,Not Adopted
156631,A894181,Return to Owner,2.0,Young,Not Adopted


In [6]:
age_adoption_contingency_table = pd.crosstab(age_adoptiion_research["age_group"], age_adoptiion_research["adoption_status"])
age_adoption_contingency_table

adoption_status,Adopted,Not Adopted
age_group,,
Young,55770,56329
Adult,14221,18397
Senior,3893,8023


In [7]:
age_groups = ["Young", "Adult", "Senior"]

for age_group in age_groups:
    total = age_adoption_contingency_table.loc[age_group].sum()
    adoption_count = age_adoption_contingency_table.loc[age_group, "Adopted"]
    adoption_count_rate = round(adoption_count/total, 4)*100.0
    print(f"The adoption rate for {age_group.lower()} dogs/cats is {adoption_count_rate}%")

The adoption rate for young dogs/cats is 49.75%
The adoption rate for adult dogs/cats is 43.6%
The adoption rate for senior dogs/cats is 32.67%


In [8]:
chi2, p, dof, expected = chi2_contingency(age_adoption_contingency_table)
print(f"chi2:    {chi2}")
print(f"p-value: {p}")
print(f"dof:     {dof}")
print(f"expected:\n{expected}")

chi2:    1471.8384693196667
p-value: 0.0
dof:     2
expected:
[[52877.25138381 59221.74861619]
 [15385.95514355 17232.04485645]
 [ 5620.79347264  6295.20652736]]


In [9]:
chi2, p, dof, expected = chi2_contingency(age_adoption_contingency_table)

# set significance level = 0.05
alpha = 0.05
if p < alpha:
    print("There is a significant relationship between age and adoption status.")
else:
    print("There is no significant relationship between age and adoption status.")

There is a significant relationship between age and adoption status.


In [10]:
fig = px.histogram(age_adoptiion_research,
             x="age_group",
             color="adoption_status",
             barmode = "group",
             title="Adoption Status by Intake Condition")

fig.update_layout(title_x=0.5,
                  xaxis=dict(title="Adoption Status"),
                  yaxis=dict(title="Counts"),
                  legend_title_text="Adoption Status",
                  height=600,
                  width=1000)

fig.update_layout(legend=dict(
    orientation="v",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))

fig.show()